In [ ]:
import pandas as pd
import numpy as np
from scipy.special import softmax, expit
from scipy.stats import norm
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.random_projection import GaussianRandomProjection
import matplotlib.pyplot as plt

## Importing MNIST from csv

In [ ]:
data = np.genfromtxt('mnist_train.csv', delimiter=',')

## Data preprocessing: create a basic unbalanced dataset

In [ ]:
N_train = 2000
N_val = 400
N_test = 1000
n_unbal = 20 

train_X, train_Y = data[:N_train,1:], data[:N_train,0].astype(int)
val_X, val_Y = data[N_train:N_train+N_val,1:], data[N_train:N_train+N_val,0].astype(int)
test_X, test_Y = data[N_train+N_val:N_train+N_val+N_test,1:], data[N_train+N_val:N_train+N_val+N_test,0].astype(int)

print(val_Y)

indices_8 = np.where(train_Y==8)
train_X_only_eight = train_X[train_Y==8]

idx_sampled = np.random.choice(train_X_only_eight.shape[0], n_unbal, replace=False)

train_X_only_eight = train_X[train_Y==8,:]
plt.imshow(train_X_only_eight[1,:].reshape(28,28))
train_X_unbal = np.vstack((
    train_X_only_eight[idx_sampled],
    train_X[train_Y != 8,:]
))

train_Y_only_eight = train_Y[train_Y==8]
train_Y_unbal = np.concatenate((
    train_Y_only_eight[idx_sampled],
    train_Y[train_Y != 8]
))


print(np.sum(train_Y_unbal==8))
print(train_X_unbal[np.where(train_Y_unbal==3)])
print(train_Y_unbal.shape)

def normalize(M):
    M = (M-min(M))/(max(M)-min(M))
    return(M)

def oneHotEncode(Y):
    onehot_encoded = []
    for value in Y:
        zero_vec = [0]*10
        zero_vec[value] = 1
        onehot_encoded.append(zero_vec)
    return np.array(onehot_encoded)

train_X_unbal = np.apply_along_axis(normalize, 1, train_X_unbal)
val_X = np.apply_along_axis(normalize, 1, val_X)
test_X = np.apply_along_axis(normalize, 1, test_X)

where_test_8 = np.where(test_Y==8)
where_test_o = np.where(test_Y!=8)

train_Y_unbal = oneHotEncode(train_Y_unbal)
val_Y = oneHotEncode(val_Y)
test_Y = oneHotEncode(test_Y)


# Helper functions

In [ ]:
def sigmoid(s):
    return expit(s)

def sigmoid_derivative(s):
    return s * (1 - s)

def one_hot_encoding(real):
    if type(real)==int:
        encode = np.zeros((10,))
        encode[int(real)] = 1
        return(np.array([encode]))
    else:
        encode = np.zeros((len(real),10))
        for i in range(len(real)):
            encode[i,real[i]] = 1
        return(np.array(encode))

def cross_entropy_derivative(pred, real):

    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss

def clip_L2_norm(vec, C):
    vec = vec/max(1, np.linalg.norm(vec)/C)
    return vec

# NN class

In [ ]:
class mnistNN:
    def __init__(self, N, D, K, L1, L2, lr):
        
        self.neurons1 = L1
        self.neurons2 = L2
        self.lr = lr

        self.w1 = np.random.randn(D, self.neurons1)
        self.b1 = np.zeros((1, self.neurons1))
        self.w2 = np.random.randn(self.neurons1, self.neurons2)
        self.b2 = np.zeros((1, self.neurons2))
        self.w3 = np.random.randn(self.neurons2, K)
        self.b3 = np.zeros((1, K))
        
        self.gw1 = np.zeros((D, self.neurons1))
        self.gb1 = np.zeros((1, self.neurons1))
        self.gw2 = np.zeros((self.neurons1, self.neurons2))
        self.gb2 = np.zeros((1, self.neurons2))
        self.gw3 = np.zeros((self.neurons2, K))
        self.gb3 = np.zeros((1, K))

    def feedforward(self, x):
        
        # layer 1
        z1 = np.dot(x, self.w1) + self.b1
        self.a1 = sigmoid(z1)
        # layer 2
        z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(z2)
        # output layer
        z3 = np.dot(self.a2, self.w3) + self.b3
        self.a3 = softmax(z3)
        
    def computegradients(self, x, y):
        
        #backwards from output layer to input
        a3_delta = cross_entropy_derivative(self.a3, y) # w3
        z2_delta = np.dot(a3_delta, self.w3.T)
        a2_delta = z2_delta * sigmoid_derivative(self.a2) # w2
        z1_delta = np.dot(a2_delta, self.w2.T)
        a1_delta = z1_delta * sigmoid_derivative(self.a1) # w1
        
        gw3 = np.dot(self.a2.T, a3_delta)
        gb3 = np.sum(a3_delta, axis=0, keepdims=True)
        gw2 = np.dot(self.a1.T, a2_delta)
        gb2 = np.sum(a2_delta, axis=0)
        gw1 = np.dot(x.T.reshape(-1,1), a1_delta)
        gb1 = np.sum(a1_delta, axis=0) 
        
        return {'gw3': gw3, 'gb3': gb3, 'gw2': gw2, 'gb2': gb2, 'gw1': gw1, 'gb1': gb1}
            
            
    def descent(self, x, y, gradients):
        
        # update gradient
        self.w3 -= self.lr * gradients['gw3']
        self.b3 -= self.lr * gradients['gb3']
        self.w2 -= self.lr * gradients['gw2']
        self.b2 -= self.lr * gradients['gb2']
        self.w1 -= self.lr * gradients['gw1']
        self.b1 -= self.lr * gradients['gb1']

    def predict(self, x):
        self.feedforward(x)
        return self.a3.argmax()
    
    def get_loss(self, x, y):
        loss = 0
        for xx,yy in zip(x, y):
            self.feedforward(xx)
            s = self.a3
            current_loss = error(s, np.array([yy]))
            loss += current_loss
        return loss/len(x)
    
    def get_acc(self, x, y):
        acc = 0
        for xx,yy in zip(x, y):
            s = self.predict(xx)
            if s == np.argmax(yy):
                acc +=1
        return acc/len(x)*100

# NON-DP MODEL (UNBALANCED)

In [ ]:
# hyperparameters
N, D = train_X_unbal.shape
L = 500
K = 10
L1 = 128
L2 = 128
lr = 0.5
batch_size = 1
epochs = 200
np.random.seed(3)

# create model
model = mnistNN(N, D, K, L1, L2, lr)

labels_per_epoch = []
loss_vec = []
acc_train_vec = []
acc_test_vec = []
acc_o_train_vec = []
acc_8_train_vec = []
acc_o_test_vec = []
acc_8_test_vec = []

# train
for e in range(epochs):
    
    idx = list(range(L))
    np.random.shuffle(idx)
    
    labels = []
    
    #store gradients across data points
    
    # loop through datapoints
    for i in idx:
        x_i, y_i = train_X_unbal[i,:], np.array([train_Y_unbal[i]])
        labels.append(np.where(train_Y_unbal[i]==1)[0].tolist()[0])
        model.feedforward(x_i)
        gradients = model.computegradients(x_i, y_i)

        model.gw1 += gradients['gw1']
        model.gb1 += gradients['gb1']
        model.gw2 += gradients['gw2']
        model.gb2 += gradients['gb2']
        model.gw3 += gradients['gw3']
        model.gb3 += gradients['gb3']
        
    #take average and update parameters
    
    gw1_avg = model.gw1/L
    gb1_avg = model.gb1/L
    gw2_avg = model.gw2/L
    gb2_avg = model.gb2/L
    gw3_avg = model.gw3/L
    gb3_avg = model.gb3/L
    
    
    model.descent(x_i, y_i, {'gw3': gw3_avg, 'gb3': gb3_avg, 'gw2': gw2_avg, 
                                    'gb2': gb2_avg, 'gw1': gw1_avg, 'gb1': gb1_avg})
    
    loss = model.get_loss(train_X_unbal, train_Y_unbal)
    loss_vec.append(loss)
    
    #reset gradients in memory
    
    model.gw1 = np.zeros((D, L1))
    model.gb1 = np.zeros((1, L1))
    model.gw2 = np.zeros((L1, L2))
    model.gb2 = np.zeros((1, L2))
    model.gw3 = np.zeros((L2, K))
    model.gb3 = np.zeros((1, K))
    
    #save results
    
    x_train_8 = train_X_unbal[where_test_8]
    y_train_8 = train_Y_unbal[where_test_8]
    x_train_o = train_X_unbal[where_test_o]
    y_train_o = train_Y_unbal[where_test_o]
    x_test_8 = test_X[where_test_8]
    y_test_8 = test_Y[where_test_8]
    x_test_o = test_X[where_test_o]
    y_test_o = test_Y[where_test_o]
    
    
    acc_train_vec.append(model.get_acc(train_X_unbal, train_Y_unbal))
    acc_test_vec.append(model.get_acc(test_X, test_Y))
    acc_o_train_vec.append(model.get_acc(x_train_o, y_train_o))
    acc_8_train_vec.append(model.get_acc(x_train_8, y_train_8))
    acc_o_test_vec.append(model.get_acc(x_test_o, y_test_o))
    acc_8_test_vec.append(model.get_acc(x_test_8, y_test_8))
         
         
    if e%20 == 1:
        x_test_8 = test_X[where_test_8]
        y_test_8 = test_Y[where_test_8]
        x_test_o = test_X[where_test_o]
        y_test_o = test_Y[where_test_o]
        print('epoch ', e)
        print('avg loss ', np.mean(loss_vec))
        print('train accuracy ', model.get_acc(train_X_unbal, train_Y_unbal))
        print('test accuracy ', model.get_acc(test_X, test_Y))
        print('len of test set other ', len(y_test_o))
        print('test accuracy other', model.get_acc(x_test_o, y_test_o))
        print('len of test set unbal ', len(y_test_8))
        print('test accuracy unbal', model.get_acc(x_test_8, y_test_8))
        print("\n")
        
        
    labels_per_epoch.append(labels)
    
    lr = lr * 0.99
    
    
suffix = "_batch" + str(L) + "_C" + str(C) + "_noisescale" + str(noise_scale) + ".npy"
np.save("loss_vec"+suffix, loss_vec)
np.save("acc_train_vec" + suffix, acc_train_vec)
np.save("acc_test_vec" + suffix, acc_test_vec)
np.save("acc_o_train_vec" + suffix, acc_o_train_vec)
np.save("acc_8_train_vec" + suffix, acc_8_train_vec)
np.save("acc_o_test_vec" + suffix, acc_o_test_vec)
np.save("acc_8_test_vec" + suffix, acc_8_test_vec)
        
print(model.get_acc(x_test_o, y_test_o))
print(model.get_acc(x_test_8, y_test_8))

In [ ]:
# hyperparameters
N, D = train_X_unbal.shape
L = 500
K = 10
L1 = 128
L2 = 128
lr = 0.5
dp = True
C = 1
noise_scale = 0.5
epochs = 200
np.random.seed(3)

# create model
modelDP = mnistNN(N, D, K, L1, L2, lr)

#storage

labels_per_epoch = []

loss_vec_DP = []
acc_train_vec_DP = []
acc_test_vec_DP = []
acc_o_train_vec_DP = []
acc_8_train_vec_DP = []
acc_o_test_vec_DP = []
acc_8_test_vec_DP = []


# train
for e in range(epochs):
    
    idx = list(range(L))
    np.random.shuffle(idx)
    
    labels = []
    
    # loop through datapoints
    for i in idx:
        x_i, y_i = train_X_unbal[i,:], np.array([train_Y_unbal[i]])
        labels.append(np.where(train_Y_unbal[i]==1)[0].tolist()[0])
        modelDP.feedforward(x_i)
        gradients = modelDP.computegradients(x_i, y_i)

        modelDP.gw1 += clip_L2_norm(gradients['gw1'], C)
        modelDP.gb1 += clip_L2_norm(gradients['gb1'], C)
        modelDP.gw2 += clip_L2_norm(gradients['gw2'], C)
        modelDP.gb2 += clip_L2_norm(gradients['gb2'], C)
        modelDP.gw3 += clip_L2_norm(gradients['gw3'], C)
        modelDP.gb3 += clip_L2_norm(gradients['gb3'], C)

    
    modelDP.gw1 = (modelDP.gw1 + np.random.normal(loc=0, scale = noise_scale**2 * C**2, size = modelDP.gw1.shape))/L
    modelDP.gb1 = (modelDP.gb1 + np.random.normal(loc=0, scale = noise_scale**2 * C**2, size = modelDP.gb1.shape))/L
    modelDP.gw2 = (modelDP.gw2 + np.random.normal(loc=0, scale = noise_scale**2 * C**2, size = modelDP.gw2.shape))/L
    modelDP.gb2 = (modelDP.gb2 + np.random.normal(loc=0, scale = noise_scale**2 * C**2, size = modelDP.gb2.shape))/L
    modelDP.gw3 = (modelDP.gw3 + np.random.normal(loc=0, scale = noise_scale**2 * C**2, size = modelDP.gw3.shape))/L
    modelDP.gb3 = (modelDP.gb3 + np.random.normal(loc=0, scale = noise_scale**2 * C**2, size = modelDP.gb3.shape))/L
        

    
    modelDP.descent(x_i, y_i, {'gw3': modelDP.gw3, 'gb3': modelDP.gb3, 'gw2': modelDP.gw2, 
                                    'gb2': modelDP.gb2, 'gw1': modelDP.gw1, 'gb1': modelDP.gb1})
    
    loss = modelDP.get_loss(train_X_unbal, train_Y_unbal)
    loss_vec_DP.append(loss)
    
    #reset gradients in memory
    
    modelDP.gw1 = np.zeros((D, L1))
    modelDP.gb1 = np.zeros((1, L1))
    modelDP.gw2 = np.zeros((L1, L2))
    modelDP.gb2 = np.zeros((1, L2))
    modelDP.gw3 = np.zeros((L2, K))
    modelDP.gb3 = np.zeros((1, K))
    
    
    #save results
    
    x_train_8 = train_X_unbal[where_test_8]
    y_train_8 = train_Y_unbal[where_test_8]
    x_train_o = train_X_unbal[where_test_o]
    y_train_o = train_Y_unbal[where_test_o]
    x_test_8 = test_X[where_test_8]
    y_test_8 = test_Y[where_test_8]
    x_test_o = test_X[where_test_o]
    y_test_o = test_Y[where_test_o]
    
    
    acc_train_vec_DP.append(modelDP.get_acc(train_X_unbal, train_Y_unbal))
    acc_test_vec_DP.append(modelDP.get_acc(test_X, test_Y))
    acc_o_train_vec_DP.append(modelDP.get_acc(x_train_o, y_train_o))
    acc_8_train_vec_DP.append(modelDP.get_acc(x_train_8, y_train_8))
    acc_o_test_vec_DP.append(modelDP.get_acc(x_test_o, y_test_o))
    acc_8_test_vec_DP.append(modelDP.get_acc(x_test_8, y_test_8))
         
    if e%20 == 1:
        
        print('epoch ', e)
        print('avg loss ', np.mean(loss_vec_DP))
        print('train accuracy ', modelDP.get_acc(train_X_unbal, train_Y_unbal))
        print('test accuracy ', modelDP.get_acc(test_X, test_Y))
        print('len of test set other ', len(y_test_o))
        print('test accuracy other', modelDP.get_acc(x_test_o, y_test_o))
        print('len of test set unbal ', len(y_test_8))
        print('test accuracy unbal', modelDP.get_acc(x_test_8, y_test_8))
        print('\n')
        
        
    labels_per_epoch.append(labels)

suffix = "_batch" + str(L) + "_C" + str(C) + "_noisescale" + str(noise_scale) + ".npy"
np.save("loss_vec_DP" + suffix, loss_vec_DP)
np.save("acc_train_vec_DP" + suffix, acc_train_vec_DP)
np.save("acc_test_vec_DP" + suffix, acc_test_vec_DP)
np.save("acc_o_train_vec_DP" + suffix, acc_o_train_vec_DP)
np.save("acc_8_train_vec_DP" + suffix, acc_8_train_vec_DP)
np.save("acc_o_test_vec_DP" + suffix, acc_o_test_vec_DP)
np.save("acc_8_test_vec_DP" + suffix, acc_8_test_vec_DP)
    
print(modelDP.get_acc(x_test_o, y_test_o))
print(modelDP.get_acc(x_test_8, y_test_8))